# Reconhecimento de Objetos com Redes Neurais Convolucionais Usando  Keras

<b> Jason Brownlee</b>

1) É preciso instalar o tensorboard para a visualização

$ pip install tensorboard

2) Após iniciar a execução abra um terminal no mesmo diretório do jupyter e lance o tensorboard com o seguinte comando:

$ tensorboard --logdir=logs/

3) O tensorboard irá informa a URL para monitorar a execução:

TensorBoard 1.7.0 at http://lccomp-Studio-XPS-8100:6006


In [ ]:
# Plot ad hoc CIFAR10 instances
from keras.datasets import cifar10
from matplotlib import pyplot
from scipy.misc import toimage
# load data
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
# create a grid of 3x3 images
for i in range(0, 9):
	pyplot.subplot(330 + 1 + i)
	pyplot.imshow(toimage(X_train[i]))
# show the plot
pyplot.show()

# Alguns comentários sobre o código


<b>Em relação ao padding na convolução</b>

    "VALID" = sem padding:

       inputs:         1  2  3  4  5  6  7  8  9  10 11 (12 13)
                      |________________|                dropped
                                     |_________________|

    "SAME" = com zero padding:

                   pad|                                      |pad
       inputs:      0 |1  2  3  4  5  6  7  8  9  10 11 12 13|0  0
                   |________________|
                                  |_________________|
                                                 |________________|

   onde: 
    entrada = 13
    kernel = 6
    Stride = 5
    

<b>Em relação ao kernel_constraint=maxnorm(3)</b>

maxnorm (m) irá, se a Norma L2 de seus pesos exceder m, escalar toda a  matriz de peso por um fator que reduza a norma para m. 

Restringir diretamente a matriz de peso é outro tipo de regularização. Se você usa um termo simples de regularização L2, penaliza os pesos maiores com sua função de perda. Com essa restrição, você regulariza diretamente. Isto parece funcionar especialmente bem em combinação com uma camada de dropout.


<b>Em relação ao pooling</b>

<i>pool_size</i>: inteiro ou tupla de 2 inteiros indicando fator de redução (vertical, horizontal). (2, 2) irá reduzir pela metade a entrada em ambas as dimensões espaciais. Se apenas um inteiro for especificado, o mesmo tamanho da janela será usado para ambas as dimensões.
 

In [ ]:
# Simple CNN model for CIFAR-10
from time import time
import numpy
from keras.utils import np_utils
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.optimizers import Nadam
from keras.optimizers import Adam

from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.callbacks import TensorBoard

from keras import backend as K
K.set_image_dim_ordering('th')



# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# load data
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

#print("Xtrain[0] ",X_train[0])
#print("Ytrain[0] ",y_train[0])


# normalize inputs from 0-255 to 0.0-1.0
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

#print("Xtrain[0] ",X_train[0])
#print("Ytrain[0] ",y_train[0])

# Create the model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(3, 32, 32), activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))

model.add(Dense(num_classes, activation='softmax'))
# Compile model
epochs =25
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
#nadam=Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
#adam=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

#model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer=nadam, metrics=['accuracy'])
print(model.summary())


tensorboard = TensorBoard(log_dir="logs/{}".format(time()))


# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=32, callbacks=[tensorboard])
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))


model.save('modelT1')

print("\nFim")

In [ ]:
from keras.models import load_model


model = load_model('modelT1')
print(model.predict(X_test[:5]))
